In [569]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
from prophet import Prophet
import timesfm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# --------------------------
# Required Packages for Models
# --------------------------
import xgboost as xgb
from lightgbm import LGBMRegressor

# PyTorch and related libraries
import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
absl_logger = logging.getLogger("absl")
cmdstanpy_logger.disabled = True
absl_logger.disabled = True

In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 26379.27it/s]


### Load data

In [ ]:
df_omie_b = pd.read_csv('../../data/df_omie_blind(in).csv')
df_omie_l = pd.read_csv('../../data/df_omie_labelled(in).csv')
filtered_cat = pd.read_csv('../../data/filtered_categories(in).csv')
unit_list = pd.read_csv('../../data/unit_list(in).csv')

In [ ]:
data = df_omie_l.merge(unit_list, on='Codigo', how='left')
codes = filtered_cat['Codigo'].unique()
data = data[data['Codigo'].isin(codes)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703248 entries, 0 to 712263
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Codigo                703248 non-null  object 
 1   Descripcion           703248 non-null  object 
 2   fechaHora             703248 non-null  object 
 3   PrecEuro              703248 non-null  float64
 4   Energia               703248 non-null  float64
 5   Descripción           403466 non-null  object 
 6   Agente                403466 non-null  object 
 7   Porcentaje_Propiedad  403466 non-null  float64
 8   Tipo_Unidad           403466 non-null  object 
 9   Zona/Frontera         403466 non-null  object 
 10  Tecnología            403466 non-null  object 
dtypes: float64(3), object(8)
memory usage: 64.4+ MB


In [ ]:
data['Codigo'] = data['Codigo'].shift(1)

In [ ]:
data['fechaHora'] = pd.to_datetime(data['fechaHora'])
data['cum_energy'] = data.groupby(['Codigo', 'fechaHora'])['Energia'].cumsum()


In [ ]:
def time_features(df: pd.DataFrame):
    df['fechaHora'] = pd.to_datetime(df['fechaHora'])
    df['date'] = df['fechaHora'].dt.date
    df['hour'] = df['fechaHora'].dt.hour
    df['day_of_week'] = df['fechaHora'].dt.dayofweek  # Monday=0, Sunday=6
    df['month'] = df['fechaHora'].dt.month
    df['day_of_month'] = df['fechaHora'].dt.day
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df.sort_values(['fechaHora', 'Codigo'], inplace=True)
    df['t'] = (df['fechaHora'] - df['fechaHora'].min()).dt.total_seconds() / 3600
    
    def add_fourier_terms(df: pd.DataFrame, period, K, time_col='t'):
        df = df.sort_values(['Codigo', 'fechaHora'])
        for k in range(1, K + 1):
            df[f'sin_{period}_{k}'] = np.sin(2 * np.pi * k * df[time_col] / period)
            df[f'cos_{period}_{k}'] = np.cos(2 * np.pi * k * df[time_col] / period)
        return df
    
    df = add_fourier_terms(df, period=24, K=3)
    
    return df

def cyclical_features(df: pd.DataFrame):
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    return df
    
def agg_features(df: pd.DataFrame):
    df['cum_energy'] = df.groupby(['Codigo', 'date'])['Energia'].cumsum()
    return df
        
def interaction_features(df: pd.DataFrame):
    df['energia_hour_sin'] = df['Energia'] * df['hour_sin']
    return df
    
def lags_features(df: pd.DataFrame):
    df.sort_values(['fechaHora'], inplace=True)
    df['PrecEuro'] = df.groupby('Codigo')['PrecEuro'].shift(24*28)
    df['lag_Energia'] = df.groupby('Codigo')['Energia'].shift(24*28)
    df['lag_Energia'] = np.log(df['lag_Energia'] + 1)
    df['lag1_Energia'] = df.groupby('Codigo')['lag_Energia'].shift(1)
    df['lag24_Energia'] = df.groupby('Codigo')['lag_Energia'].shift(24)
    return df
    
def rolling_mean_features(df: pd.DataFrame):
    df['roll24_mean_Energia'] = df.groupby('Codigo')['Energia'] \
        .transform(lambda x: x.rolling(window=24, min_periods=1).mean())
    return df
    

In [ ]:
def feature_engineering(data: pd.DataFrame):
    data = time_features(data)
    data = cyclical_features(data)
    data = agg_features(data)
    data = interaction_features(data)
    data = lags_features(data)
    data = rolling_mean_features(data)
    return data

In [ ]:
data = feature_engineering(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703248 entries, 1 to 712104
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Codigo                703247 non-null  object        
 1   Descripcion           703248 non-null  object        
 2   fechaHora             703248 non-null  datetime64[ns]
 3   PrecEuro              493583 non-null  float64       
 4   Energia               703248 non-null  float64       
 5   Descripción           403466 non-null  object        
 6   Agente                403466 non-null  object        
 7   Porcentaje_Propiedad  403466 non-null  float64       
 8   Tipo_Unidad           403466 non-null  object        
 9   Zona/Frontera         403466 non-null  object        
 10  Tecnología            403466 non-null  object        
 11  cum_energy            703247 non-null  float64       
 12  date                  703248 non-null  object        
 13  hour

In [ ]:
data = data.sort_values(['fechaHora', 'Codigo'])

# Create a log-transformed and first-differenced (stationary) series.
data['Energia_stationary'] = data['Energia'].diff()

### Missing Values

Due to lags, we have many missing values at the beginning of each codigo. As it represents a large proportion of the df, I will impute by the mean per hour. This might cause leakages for the first records but shouldn't impact the model.

In [ ]:
nan_counts = data.isna().sum()
nan_counts = nan_counts[nan_counts > 1]
print(nan_counts)

PrecEuro                209665
Descripción             299782
Agente                  299782
Porcentaje_Propiedad    299782
Tipo_Unidad             299782
Zona/Frontera           299782
Tecnología              299782
lag_Energia             209665
lag1_Energia            209977
lag24_Energia           217153
dtype: int64


Let's drop Technologia, Zona/Frontera, Tipo_Unidad, Porcentaje_Propiedad, Agente and Descripción as their feature importance is close to 0.

In [ ]:
data.drop(columns=['Descripción', 'Agente', 'Porcentaje_Propiedad', 'Tipo_Unidad', 'Zona/Frontera', 'Tecnología'], inplace=True)

In [ ]:
def impute_codigo_hour(df: pd.DataFrame):
    """
    Impute missing values in selected columns using the mean computed per Codigo and per hour.
    """
    df = df.copy()

    df['fechaHora'] = pd.to_datetime(df['fechaHora'])

    cols_to_impute = ['PrecEuro', 'lag_Energia', 'lag1_Energia', 'lag24_Energia']

    for col in cols_to_impute:
        df[col] = df.groupby(['Codigo', 'hour'])[col].transform(lambda x: x.fillna(x.mean()))
        
    return df

In [ ]:
data = impute_codigo_hour(data)

In [ ]:
data.dropna(inplace=True)

### Ordinal Encoding

In [ ]:
encoder = OrdinalEncoder()
cat_cols=data.select_dtypes(include=['object']).columns
data[cat_cols] = encoder.fit_transform(data[cat_cols])

## Models

### TimesFM

In [ ]:
class TimesFMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TimesFMModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
def prepare_timesfm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    return X_train_tensor, y_train_tensor, X_test_tensor

In [ ]:
TIMESFM_MODEL_PARAMS = {
    'hidden_dim': 100,
    'output_dim': 1,
}

TIMESFM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [ ]:
def train_timesfm(
    X_train_tensor: torch.Tensor, 
    y_train_tensor: torch.Tensor,
    TIMESFM_MODEL_PARAMS: dict,
    TIMESFM_TRAIN_PARAMS: dict,
    epochs: int = 150
    ) -> TimesFMModel:

    timesfm_model = TimesFMModel(input_dim=len(FEATURES), **TIMESFM_MODEL_PARAMS)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(timesfm_model.parameters(), **TIMESFM_TRAIN_PARAMS)
    
    # Train the model.
    timesfm_model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = timesfm_model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
    return timesfm_model
    
def evaluate_timesfm(timesfm_model: TimesFMModel, X_test_tensor: torch.Tensor) -> np.ndarray:
    timesfm_model.eval()
    with torch.no_grad():
        y_pred_timesfm = timesfm_model(X_test_tensor).numpy().flatten()
        
    return y_pred_timesfm

### LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # batch_first=True makes input shape (batch, seq, feature)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden and cell states with zeros (and send to same device as x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        # Use the output from the final time step
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
def prepare_lstm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    selected_features = [
        'lag1_Energia', 'lag24_Energia', 'roll24_mean_Energia',
        'hour', 'day_of_week', 'day_of_month', 'month', 'is_weekend',
        'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
        'PrecEuro', 'cum_energy'
    ]
    X_train_selected = X_train[selected_features].values  # shape: (num_samples, num_features)
    X_test_selected  = X_test[selected_features].values

    # Add a time dimension (sequence length = 1)
    X_train_lstm = torch.tensor(X_train_selected, dtype=torch.float32).unsqueeze(1)  # shape: (samples, 1, num_features)
    X_test_lstm  = torch.tensor(X_test_selected, dtype=torch.float32).unsqueeze(1)
    y_train_lstm = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    
    return X_train_lstm, y_train_lstm, X_test_lstm

In [ ]:
LSTM_MODEL_PARAMS = {
    'hidden_size': 50,
    'num_layers': 1,
    'output_size': 1
}

LSTM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [ ]:
def train_lstm(
    X_train: torch.Tensor, 
    y_train: torch.Tensor, 
    LSTM_MODEL_PARAMS: dict,
    LSTM_TRAIN_PARAMS: dict,
    epochs: int=150
    ) -> LSTMModel:
    
    input_size = X_train.shape[2]
    lstm_model = LSTMModel(input_size, **LSTM_MODEL_PARAMS)

    criterion_lstm = nn.MSELoss()
    optimizer_lstm = optim.Adam(lstm_model.parameters(), **LSTM_TRAIN_PARAMS)
    
    lstm_model.train()
    for epoch in range(epochs):
        optimizer_lstm.zero_grad()
        outputs = lstm_model(X_train)
        loss = criterion_lstm(outputs, y_train)
        loss.backward()
        optimizer_lstm.step()
        
    return lstm_model
    
    
def evaluate_lstm(lstm_model: LSTMModel, X_test_lstm: torch.Tensor) -> np.ndarray:
    lstm_model.eval()
    with torch.no_grad():
        y_pred_lstm = lstm_model(X_test_lstm).numpy().flatten()
    return y_pred_lstm

### XGB

In [ ]:
XGB_PARAMS = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0,
    'booster': 'gbtree'
}

In [ ]:
def train_xgb(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    XGB_PARAMS: dict) -> xgb.XGBRegressor:    
    xgb_reg = xgb.XGBRegressor(**XGB_PARAMS)
    xgb_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )
    return xgb_reg
    
def evaluate_xgb(xgb_reg: xgb.XGBRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return xgb_reg.predict(X_test)
    

### LGBM

In [ ]:
LGBM_PARAMS = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 3
}

In [ ]:
def train_lgbm(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    LGBM_PARAMS: dict
    ) -> LGBMRegressor:    
    lgbm_reg = LGBMRegressor(**LGBM_PARAMS)
    lgbm_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)]
    )
    return lgbm_reg
    
def evaluate_lgbm(lgbm_reg: LGBMRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return lgbm_reg.predict(X_test)

### Prophet

In [ ]:
def prepare_prophet(train: pd.DataFrame) -> pd.DataFrame:
    train_prophet = train.reset_index()
    train_prophet['ds'] = pd.to_datetime(train_prophet['fechaHora'])

    train_prophet = train_prophet.rename(columns={TARGET: 'y'})

    return train_prophet[['ds', 'y', 'PrecEuro', 'cum_energy']]

def train_prophet(train_prophet: pd.DataFrame) -> Prophet:

    prophet_model = Prophet()
    prophet_model.add_regressor('PrecEuro')
    prophet_model.add_regressor('cum_energy')

    # Fit the model
    prophet_model.fit(train_prophet)
    
    return prophet_model

def evaluate_prophet(prophet_model: Prophet, test: pd.DataFrame) -> np.ndarray:

    test_prophet = test.reset_index()
    test_prophet['ds'] = pd.to_datetime(test_prophet['fechaHora'])
    future = test_prophet[['ds', 'PrecEuro', 'cum_energy']]

    # Forecast
    forecast = prophet_model.predict(future)
    return forecast['yhat'].values

## Benchmark

### Custom Rolling Window CV

In [ ]:
def custom_rolling_window_cv(data: pd.DataFrame, initial_train_window: int, forecast_horizon: int, step: int):
    """
    Custom rolling window cross-validation.
    """
    n = len(data)
    train_end = initial_train_window  
    while (train_end + forecast_horizon) <= n:
        train_idx = list(range(0, train_end))
        test_idx = list(range(train_end, train_end + forecast_horizon))
        yield train_idx, test_idx
        train_end += step

In [ ]:
nunique_codes = data['Codigo'].nunique()
INITIAL_TRAIN_WINDOW = 24*28*nunique_codes
FORECAST_HORIZON = 24*28*nunique_codes
STEP = 24*7*nunique_codes

EXCLUDED_COLS = ['fechaHora', 'Energia']
FEATURES = [col for col in data.columns if col not in EXCLUDED_COLS]
TARGET = 'Energia_stationary'

model_names = ['TimesFM', 'XGB', 'LGBM', 'Prophet', 'LSTM']
results = {model: {'mae': [], 'mape': []} for model in model_names}

In [ ]:
def mae_score(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape_score(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.abs(y_true+ 1e-3)))

In [ ]:
for fold, (train_idx, test_idx) in enumerate(custom_rolling_window_cv(data, INITIAL_TRAIN_WINDOW, FORECAST_HORIZON, STEP)):
    print(f"\n===== Fold {fold} =====")
    
    # Split the data into train and test folds
    train = data.iloc[train_idx]
    test = data.iloc[test_idx]
    
    # Common splits for models that require X and y inputs:
    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test  = test[FEATURES]
    y_test  = test[TARGET]
    
    # --------------------------
    if 'Prophet' in model_names:
        data_prophet = prepare_prophet(train)
        prophet_model = train_prophet(data_prophet)
        y_pred_prophet = evaluate_prophet(prophet_model, test)
        
        mae_prophet = mae_score(y_test, y_pred_prophet)
        mape_prophet = mape_score(y_test, y_pred_prophet)
        
        results['Prophet']['mae'].append(mae_score(y_test, y_pred_prophet))
        results['Prophet']['mape'].append(mape_prophet)
        
        print(f"Prophet    --> MAE: {mae_prophet:.4f}, MAPE: {mape_prophet:.4f}")
    
    # --------------------------
    if 'XGB' in model_names:
        xgb_model = train_xgb(X_train, y_train, X_test, y_test, XGB_PARAMS)
        y_pred_xgb = evaluate_xgb(xgb_model, X_test)
        
        mae_xgb = mae_score(y_test, y_pred_xgb)
        mape_xgb = mape_score(y_test, y_pred_xgb)
        
        results['XGB']['mae'].append(mae_xgb)
        results['XGB']['mape'].append(mape_xgb)
        
        print(f"XGBoost    --> MAE: {mae_xgb:.4f}, MAPE: {mape_xgb:.4f}")
    
    # --------------------------
    if 'LGBM' in model_names:
        lgbm_model = train_lgbm(X_train, y_train,  X_test, y_test, LGBM_PARAMS)
        y_pred_lgbm = evaluate_lgbm(lgbm_model, X_test)

        mae_lgbm = mae_score(y_test, y_pred_lgbm)
        mape_lgbm = mape_score(y_test, y_pred_lgbm)

        results['LGBM']['mae'].append(mae_lgbm)
        results['LGBM']['mape'].append(mape_lgbm)

        print(f"LightGBM   --> MAE: {mae_lgbm:.4f}, MAPE: {mape_lgbm:.4f}")

    # --------------------------
    if 'TimesFM' in model_names:
        X_train_timesfm, y_train_timesfm, X_test_timesfm = prepare_timesfm(X_train, y_train, X_test)  
        timesfm_model = train_timesfm(X_train_timesfm, y_train_timesfm, TIMESFM_MODEL_PARAMS, TIMESFM_TRAIN_PARAMS)
        y_pred_timesfm = evaluate_timesfm(timesfm_model, X_test_timesfm)
        
        mae_timesfm = mae_score(y_test, y_pred_timesfm)
        mape_timesfm = mape_score(y_test, y_pred_timesfm)
        
        results['TimesFM']['mae'].append(mae_timesfm)
        results['TimesFM']['mape'].append(mape_timesfm)
        
        print(f"TimesFM   --> MAE: {mae_timesfm:.4f}, MAPE: {mape_timesfm:.4f}")
    
    # --------------------------
    # LSTM Model (PyTorch)
    # --------------------------
    if 'LSTM' in model_names:
        X_train_lstm, y_train_lstm, X_test_timesfm = prepare_lstm(X_train, y_train, X_test)
        lstm_model = train_lstm(X_train_lstm, y_train_lstm, LSTM_MODEL_PARAMS, LSTM_TRAIN_PARAMS)
        y_pred_lstm = evaluate_lstm(lstm_model, X_test_timesfm)
        
        mae_lstm = mae_score(y_test, y_pred_lstm)
        mape_lstm = mape_score(y_test, y_pred_lstm)
        
        results['LSTM']['mae'].append(mae_lstm)
        results['LSTM']['mape'].append(mape_lstm)
        
        print(f"LSTM       --> MAE: {mae_lstm:.4f}, MAPE: {mape_lstm:.4f}")
    
# --------------------------
# STEP 5: Aggregate and Display Results
# --------------------------
benchmark_results = []
for model in model_names:
    avg_mae = np.mean(results[model]['mae'])
    avg_mape = np.mean(results[model]['mape'])
    benchmark_results.append({
        'Model': model,
        'MAE': avg_mae,
        'MAPE': avg_mape
    })

results_df = pd.DataFrame(benchmark_results)
print("\n===== Benchmark Results =====")
print(results_df)


===== Fold 0 =====


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Prophet    --> MAE: 131.3310, MAPE: 20855.1568


XGBoostError: [20:09:13] /Users/runner/work/xgboost/xgboost/src/data/data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000030763858c dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000003077964ec xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 2776
  [bt] (2) 3   libxgboost.dylib                    0x0000000307795888 xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 464
  [bt] (3) 4   libxgboost.dylib                    0x000000030764fbc4 XGDMatrixSetInfoFromInterface + 228
  [bt] (4) 5   libffi.8.dylib                      0x000000010830804c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x0000000108305834 ffi_call_int + 1404
  [bt] (6) 7   _ctypes.cpython-311-darwin.so       0x0000000106fd8134 _ctypes_callproc 

In [ ]:
XGB_PARAMS = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0,
    'booster': 'gbtree'
}

# Convert 'fechaHora' column to datetime
data['fechaHora'] = pd.to_datetime(data['fechaHora'])

# Split the data such that the test set represents the last 3 months
split_date = pd.to_datetime('2024-05-01')
train = data[data['fechaHora'] < split_date]
test = data[data['fechaHora'] >= split_date]

# Common splits for models that require X and y inputs:
X_train = train[FEATURES]
y_train = train[TARGET]
X_test  = test[FEATURES]
y_test  = test[TARGET]

xgb_model = train_xgb(X_train, y_train, X_test, y_test, XGB_PARAMS)
y_pred_xgb = evaluate_xgb(xgb_model, X_test)

mae_xgb = mae_score(y_test, y_pred_xgb)
mape_xgb = mape_score(y_test, y_pred_xgb)

print(f"XGBoost    --> MAE: {mae_xgb:.4f}, MAPE: {mape_xgb:.4f}")

XGBoost    --> MAE: 5.3114, MAPE: 7.1780


In [ ]:
booster = xgb_model.get_booster()
gain_importances = booster.get_score(importance_type='gain')
cover_importances = booster.get_score(importance_type='cover')
print("Gain-based importances:", gain_importances)
print("Cover-based importances:", cover_importances)


Gain-based importances: {'Codigo': 15071271.0, 'PrecEuro': 3375221.5, 'cum_energy': 18128722.0, 'date': 15013311.0, 'day_of_week': 12675636.0, 'day_of_month': 26607256.0, 'sin_24_1': 8660682.0, 'sin_24_3': 5884109.5, 'lag_Energia': 13829793.0, 'lag1_Energia': 15316444.0, 'lag24_Energia': 13615051.0, 'roll24_mean_Energia': 40523412.0, 'Energia_stationary': 161339488.0}
Cover-based importances: {'Codigo': 1625.7435302734375, 'PrecEuro': 158.0, 'cum_energy': 17416.72265625, 'date': 1014.9596557617188, 'day_of_week': 1021.686767578125, 'day_of_month': 865.9361572265625, 'sin_24_1': 424.2727355957031, 'sin_24_3': 426.79998779296875, 'lag_Energia': 514.4390258789062, 'lag1_Energia': 460.0, 'lag24_Energia': 14483.6611328125, 'roll24_mean_Energia': 23202.5390625, 'Energia_stationary': 83649.890625}


In [ ]:
xgb_model.feature_importances_


array([0.04305581, 0.        , 0.00964238, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.05179037,
       0.04289022, 0.        , 0.03621192, 0.        , 0.07601196,
       0.        , 0.        , 0.02474195, 0.        , 0.        ,
       0.        , 0.0168098 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03950914, 0.04375622,
       0.03889566, 0.11576782, 0.4609168 ], dtype=float32)

In [ ]:
booster = xgb_model.get_booster()
gain_importances = booster.get_score(importance_type='gain')
print("Importancias basadas en gain:", gain_importances)


Importancias basadas en gain: {'Codigo': 15071271.0, 'PrecEuro': 3375221.5, 'cum_energy': 18128722.0, 'date': 15013311.0, 'day_of_week': 12675636.0, 'day_of_month': 26607256.0, 'sin_24_1': 8660682.0, 'sin_24_3': 5884109.5, 'lag_Energia': 13829793.0, 'lag1_Energia': 15316444.0, 'lag24_Energia': 13615051.0, 'roll24_mean_Energia': 40523412.0, 'Energia_stationary': 161339488.0}


In [ ]:
# Create a DataFrame with feature names and their importance scores
importance_df = pd.DataFrame({
    'Feature': FEATURES,
    'Importance': xgb_model.feature_importances_
})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the sorted DataFrame
print(importance_df)

                 Feature  Importance
32    Energia_stationary    0.460917
31   roll24_mean_Energia    0.115768
14          day_of_month    0.076012
9             cum_energy    0.051790
29          lag1_Energia    0.043756
0                 Codigo    0.043056
10                  date    0.042890
28           lag_Energia    0.039509
30         lag24_Energia    0.038896
12           day_of_week    0.036212
17              sin_24_1    0.024742
21              sin_24_3    0.016810
2               PrecEuro    0.009642
6            Tipo_Unidad    0.000000
3            Descripción    0.000000
27      energia_hour_sin    0.000000
26               dow_cos    0.000000
25               dow_sin    0.000000
24              hour_cos    0.000000
23              hour_sin    0.000000
22              cos_24_3    0.000000
19              sin_24_2    0.000000
20              cos_24_2    0.000000
7          Zona/Frontera    0.000000
18              cos_24_1    0.000000
1            Descripcion    0.000000
1

## Hyperparameter Tuning

In [ ]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

# ============================================
# Assumed pre-defined functions and variables:
# --------------------------------------------
# - data: your full DataFrame.
# - FEATURES, TARGET: list of feature column names and target column name.
# - tss: a time series cross-validation splitter.
# - mae_score: a function to compute MAE.
#
# For each model, you must have implemented:
#   Prophet: prepare_prophet(), train_prophet() [or inline training], evaluate_prophet()
#   XGB: train_xgb(), evaluate_xgb()
#   LGBM: train_lgbm(), evaluate_lgbm()
#   TimesFM: prepare_timesfm(), train_timesfm(), evaluate_timesfm()
#   LSTM: prepare_lstm(), train_lstm(), evaluate_lstm()
# ============================================

def objective(trial):
    # Choose which model to tune in this trial.
    model_name = trial.suggest_categorical("model", ["Prophet", "XGB", "LGBM", "TimesFM", "LSTM"])
    
    fold_scores = []  # To store metric (e.g., MAE) for each CV fold
    
    for train_idx, test_idx in tss.split(data):
        # Split the data for this fold
        train = data.iloc[train_idx]
        test = data.iloc[test_idx]
        
        # For models that require X and y explicitly.
        X_train = train[FEATURES]
        y_train = train[TARGET]
        X_test = test[FEATURES]
        y_test = test[TARGET]
        
        if model_name == "Prophet":
            # Prepare Prophet data
            train_prophet = prepare_prophet(train)
            # Hyperparameters (regularization via changepoint and seasonality scales)
            cp_scale = trial.suggest_float("prophet_changepoint_prior_scale", 0.001, 0.5, log=True)
            seas_scale = trial.suggest_float("prophet_seasonality_prior_scale", 0.01, 10.0, log=True)
            
            # Initialize and train Prophet with extra regressors
            from prophet import Prophet  # or fbprophet depending on your package
            prophet_model = Prophet(changepoint_prior_scale=cp_scale,
                                    seasonality_prior_scale=seas_scale)
            prophet_model.add_regressor('PrecEuro')
            prophet_model.add_regressor('cum_energy')
            prophet_model.fit(train_prophet)
            
            # Evaluate
            y_pred = evaluate_prophet(prophet_model, test)
            
        elif model_name == "XGB":
            # Hyperparameter search space for XGBoost
            xgb_params = {
                "n_estimators": trial.suggest_int("xgb_n_estimators", 100, 1000),
                "max_depth": trial.suggest_int("xgb_max_depth", 3, 10),
                "learning_rate": trial.suggest_float("xgb_learning_rate", 0.001, 0.3, log=True),
                "reg_alpha": trial.suggest_float("xgb_reg_alpha", 0.0, 10.0),
                "reg_lambda": trial.suggest_float("xgb_reg_lambda", 0.0, 10.0),
                "subsample": trial.suggest_float("xgb_subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0)
            }
            xgb_model = train_xgb(X_train, y_train, X_test, y_test, xgb_params)
            y_pred = evaluate_xgb(xgb_model, X_test)
            
        elif model_name == "LGBM":
            # Hyperparameter search space for LightGBM
            lgbm_params = {
                "n_estimators": trial.suggest_int("lgbm_n_estimators", 100, 1000),
                "num_leaves": trial.suggest_int("lgbm_num_leaves", 20, 150),
                "learning_rate": trial.suggest_float("lgbm_learning_rate", 0.001, 0.3, log=True),
                "reg_alpha": trial.suggest_float("lgbm_reg_alpha", 0.0, 10.0),
                "reg_lambda": trial.suggest_float("lgbm_reg_lambda", 0.0, 10.0),
                "min_data_in_leaf": trial.suggest_int("lgbm_min_data_in_leaf", 10, 100)
            }
            lgbm_model = train_lgbm(X_train, y_train, X_test, y_test, lgbm_params)
            y_pred = evaluate_lgbm(lgbm_model, X_test)
            
        elif model_name == "TimesFM":
            # Prepare TimesFM data (assumed to have its own preparation method)
            X_train_tf, y_train_tf, X_test_tf = prepare_timesfm(X_train, y_train, X_test)
            # Hyperparameters for TimesFM model (example values)
            timesfm_model_params = {
                "learning_rate": trial.suggest_float("timesfm_learning_rate", 0.001, 0.1, log=True),
                "reg_alpha": trial.suggest_float("timesfm_reg_alpha", 0.0, 10.0),
                "reg_lambda": trial.suggest_float("timesfm_reg_lambda", 0.0, 10.0)
            }
            timesfm_train_params = {
                "epochs": trial.suggest_int("timesfm_epochs", 10, 200)
            }
            timesfm_model = train_timesfm(X_train_tf, y_train_tf, timesfm_model_params, timesfm_train_params)
            y_pred = evaluate_timesfm(timesfm_model, X_test_tf)
            
        elif model_name == "LSTM":
            # Prepare LSTM data (using your own prepare_lstm)
            X_train_lstm, y_train_lstm, X_test_lstm = prepare_lstm(X_train, y_train, X_test)
            # Hyperparameter search space for LSTM (e.g., architecture + training reg params)
            lstm_model_params = {
                "num_layers": trial.suggest_int("lstm_num_layers", 1, 3),
                "hidden_size": trial.suggest_int("lstm_hidden_size", 16, 128),
                "dropout": trial.suggest_float("lstm_dropout", 0.0, 0.5)
            }
            lstm_train_params = {
                "learning_rate": trial.suggest_float("lstm_learning_rate", 1e-5, 1e-2, log=True),
                "weight_decay": trial.suggest_float("lstm_weight_decay", 0.0, 0.1),
                "epochs": trial.suggest_int("lstm_epochs", 10, 200)
            }
            lstm_model = train_lstm(X_train_lstm, y_train_lstm, lstm_model_params, lstm_train_params)
            y_pred = evaluate_lstm(lstm_model, X_test_lstm)
        
        # Calculate the evaluation metric (e.g., MAE) for this fold
        fold_mae = mae_score(y_test, y_pred)
        fold_scores.append(fold_mae)
    
    # Return the average score across folds (Optuna minimizes the objective)
    return np.mean(fold_scores)


if __name__ == "__main__":
    # Set up Optuna with a TPE sampler (using a seed for reproducibility)
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # Run the optimization (adjust n_trials as needed)
    study.optimize(objective, n_trials=50)
    
    # Print out the best parameters and corresponding score
    print("Best trial:")
    best_trial = study.best_trial
    print(f"  MAE: {best_trial.value:.4f}")
    print("  Hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")


[I 2025-02-13 19:05:59,293] A new study created in memory with name: no-name-7cc3614a-041c-4ff7-9bd1-2be301f26054
[I 2025-02-13 19:06:03,647] Trial 0 finished with value: 4.236947425301009 and parameters: {'model': 'XGB', 'xgb_n_estimators': 240, 'xgb_max_depth': 3, 'xgb_learning_rate': 0.13983740016490973, 'xgb_reg_alpha': 6.011150117432088, 'xgb_reg_lambda': 7.080725777960454, 'xgb_subsample': 0.5102922471479012, 'xgb_colsample_bytree': 0.9849549260809971}. Best is trial 0 with value: 4.236947425301009.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True t

KeyboardInterrupt: 